Om de cel hier onder te runnen moet er een package geinstalleerd worden via anaconda prompt. Deze package zorgt ervoor dat we functies uit andere scripts kunnen gerbuiken in dit script.
<break>
Tik het volgende in anaconda prompt:
<br \>
**pip install ipynb**

In [1]:
%%capture
import pandas as pd
import numpy as np
from numpy.linalg import inv
from numpy.linalg import det
from pandas_datareader import data as wb
import matplotlib.pyplot as plt
from ipynb.fs.full.Dataset import getdata
from ipynb.fs.full.Dataset import getreturns
from scipy.optimize import minimize
import warnings
warnings.filterwarnings('ignore')
import scipy.stats as st
from scipy.special import gammaln

###### Hier onder wordt de dataset verkregen. Ik gebruik 2010 tot en met 2015 als mijn in-sample data.
<break>
Hiervoor gebruik een functie die in een ander script (Dataset) is geschreven

In [2]:
df = getreturns()
mInSampleReturns = df.loc[:"2015"]
mOutSampleReturns = df.loc["2016":]

Hier schat ik de garch modellen met student-t verdeling

In [3]:
def Parameters_GARCH_Student(vReturns):
    dOmega = 0.1
    dAlpha = 0.1
    dBeta = 0.8
    dNu = 4.2
    vTheta = np.array([dOmega, dAlpha, dBeta, dNu])
    
    def LL_GARCH_Student(vTheta,returns):
        dOmega = vTheta[0]
        dAlpha = vTheta[1]
        dBeta  = vTheta[2]
        dNu = vTheta[3]
        iT=len(returns)
        vH=np.zeros(iT)
        
        for t in range(iT):
            if t == 0:
                vH[t] = np.var(returns) 
            else:
                vH[t] = dOmega + dAlpha*returns[t-1]**2 + dBeta * vH[t-1]    
        
        vLogPdf = gammaln( (dNu+1)/2 )-gammaln( dNu/2 ) - 0.5*np.log( np.pi*(dNu-2)*vH[1:] ) \
                - 0.5 * (dNu+1) * np.log( 1 + ( vReturns[1:]**2 / ( (dNu-2)*vH[1:] ) ) )
        return -np.sum(vLogPdf)
    
    def Optimizer(returns, initials, function, bnds):
        result = minimize(function, initials, args=(returns), \
                          options ={'eps':1e-09, 'disp': True, 'maxiter':200}, method='SLSQP',bounds=bnds)
        return result
    bounds = ((0, 1), (0, 1), (0, 1), (2.1,50))
    result=Optimizer(vReturns, vTheta, LL_GARCH_Student, bounds)
    return result.x, -result.fun, result.success

In [4]:
(parameter1_Stud,likelihood1_Stud,success1_Stud)=Parameters_GARCH_Student(np.array(mInSampleReturns.iloc[:,0]))
print(parameter1_Stud)
print(likelihood1_Stud)
print(success1_Stud)

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 2492.9007501403294
            Iterations: 16
            Function evaluations: 107
            Gradient evaluations: 16
[ 0.04129691  0.09040065  0.89042978  6.9246734 ]
-2492.9007501403294
True


In [5]:
(parameter2_Stud,likelihood2_Stud,success2_Stud)=Parameters_GARCH_Student(np.array(mInSampleReturns.iloc[:,1]))
print(parameter2_Stud)
print(likelihood2_Stud)
print(success2_Stud)

Optimization terminated successfully.    (Exit mode 0)
            Current function value: 2217.307544488237
            Iterations: 16
            Function evaluations: 108
            Gradient evaluations: 16
[ 0.02623554  0.09365917  0.89023515  6.98573207]
-2217.307544488237
True


In [6]:
def ComputeSigma2GARCH(vTheta1,vTheta2,mReturns):
    iT = mReturns.shape[1]
    iDimension = mReturns.shape[0]
    mH = np.zeros((iDimension,iT))
    dOmega1 = vTheta1[0]
    dOmega2 = vTheta2[0]
    dAlpha1 = vTheta1[1]
    dAlpha2 = vTheta2[1]
    dBeta1 = vTheta1[2]
    dBeta2 = vTheta2[2]
    for t in range(iT):
        if t==0:
            mH[0,t]=np.var(mReturns[0,:])
            mH[1,t]=np.var(mReturns[1,:])
        else:
            mH[0,t]=dOmega1 + dAlpha1 * mReturns[0,t-1]**2 + dBeta1 * mH[0,t-1]
            mH[1,t]=dOmega2 + dAlpha2 * mReturns[1,t-1]**2 + dBeta2 * mH[1,t-1]
    return mH

#### Hier schat ik de waardes van de sigma's over tijd voor alle 2 de tijdreeksen

In [7]:
mSigma2=ComputeSigma2GARCH(parameter1_Stud,parameter2_Stud,np.array(mInSampleReturns).T)

Hier voer ik de PIT. De marginal density is student-t verdeeld met mean=0 and variance=$\sigma_t^2$
Met de functie st.t.cdf zorg ik ervoor dat de marginal cdf van elke $y_t$ wordt berekend. 

In [8]:
mReturns=np.array(mInSampleReturns).T
mScale=np.sqrt(mSigma2)

vUt1=st.t.cdf(mReturns[0,:], parameter1_Stud[3], loc=np.zeros(mReturns.shape[1]), \
              scale=np.sqrt((parameter1_Stud[3]-2)/parameter1_Stud[3]*mScale[0,:]))
vUt2=st.t.cdf(mReturns[1,:], parameter2_Stud[3], loc=np.zeros(mReturns.shape[1]), \
              scale=np.sqrt((parameter2_Stud[3]-2)/parameter2_Stud[3]*mScale[1,:]))
vUt1=vUt1.reshape((1,len(mInSampleReturns)))
vUt2=vUt2.reshape((1,len(mInSampleReturns)))
mUt=np.concatenate((vUt1, vUt2), axis=0)
print(np.min(mUt,axis=1))
print(np.max(mUt,axis=1))

[ 0.00083545  0.00097807]
[ 0.9999183   0.99973922]


## Copula part

Vanaf hieronder zal ik de copula LL gaan uitwerken. Eerst moet ik de PIT transformatie toepassen op de returns. Hierbij moet ik de kans berekenen dat:
$$ P(Y_t\leq y_t)=u_t \quad Y_t\sim \text{t}(0,\sigma_t^2, \nu) $$
Waarbij $Y_t$ een stochast is en $y_t$ de realisatie.

Met de $u_t$ gaan we de eerste inverse cdf toepassen op de marginale copula density met mean 0 and std=1. In andere woorden we willen:
$x_{it}=t^{-1}_\nu(u_{it})$

De marginale copula density met student-t distribution ziet er als volgt uit:
$$ t_\nu(x)=\frac{\Gamma\left(\frac{\nu_c+1}{2}\right)}{\Gamma\left(\frac{\nu_c}{2}\right)\nu^{1/2}\pi^{1/2}}\left(1+\frac{x^2}{\nu_c}\right)^{-\frac{\nu_c+1}{2}}$$
Dit is de standaard student-t verdeling. Ik maak gebruik van de functie st.ppf() om de inverse cdf van de bijbehorende $u_t$ te bepalen.

De log-likelihood die hierbij gemaximaliseerd word is als volgt gedefinieerd:

$$\log\left(\Gamma\left(\frac{\nu+d}{2}\right)\right)+(d-1)\log\left(\Gamma\left(\frac{\nu}{2}\right)\right)-d\log\left(\Gamma\left(\frac{\nu+1}{2}\right)\right)-\frac{1}{2}\log|R| - \frac{\nu+d}{2}\log\left(1+\frac{x'R^{-1}x}{\nu}\right)+\frac{\nu+1}{2}\log\left(1+\frac{x_1^2}{\nu}\right)+\frac{\nu+1}{2}\log\left(1+\frac{x_2^2}{\nu}\right)$$

- x is gedefinieerd als:
$$x=\begin{pmatrix} x_{1}\\ x_{2}\end{pmatrix}$$

- d is de dimensie. In dit geval gelijk aan 2

- $R$ is als volgt gedefinieerd:

$$R=\begin{pmatrix} 1 & \rho \\ \rho & 1 \end{pmatrix}$$
<break>
De parameters die we optimaliseren zijn $\rho,\nu$

In [9]:
def Parameters_Copula_Student(mReturns,mUt):
    dRho12 = 0.2
    dNu = 4.1
    vTheta = np.array([dRho12, dNu])
    
    def LL_Copula_Student(vTheta,mReturns,mUt):
        dRho12 = vTheta[0]
        dNu = vTheta[1]
        iT = mReturns.shape[1]
        iDimension = len(mReturns)
        mQuantile= st.t.ppf(mUt, dNu, loc=np.zeros(mUt.shape), scale=np.ones(mUt.shape))
        mR = np.ones((iDimension,iDimension))
        mR[1,0] = dRho12
        mR[0,1] = dRho12
        
        dSum=0
        
        for t in range(1,iT):
            vX = mQuantile[:,t]
            vX = vX.reshape((iDimension,1))
            dLogf1=st.t.logpdf(vX[0],dNu)
            dLogf2=st.t.logpdf(vX[1],dNu)
#             dLogLikelihood = gammaln((dNu+2)/2) - gammaln(dNu/2) - 0.5*np.log((np.pi*dNu)**2) - 0.5 * np.log(det(mR)) \
#                             - (dNu+2)/2 * np.log(1+(vX.T @ inv(mR) @ vX)/dNu) - dLogf1-dLogf2
            dLogLikelihood = gammaln((dNu+2)/2) + gammaln(dNu/2) - iDimension * gammaln((dNu+1)/2) - 1/2 * np.log(det(mR)) \
                             -(dNu+iDimension)/2 *np.log(1+(vX.T@inv(mR)@vX)/dNu) + (dNu+1)/2 * np.log(1+vX[0]**2/dNu) + \
                             (dNu+1)/2* np.log(1+vX[1]**2/dNu)             
                
            dSum += np.asscalar(dLogLikelihood)
    
        return -dSum
    
    def Optimizer(returns, mUt, initials, function, bnds):
        result = minimize(function, initials, args=(returns,mUt), \
                          options ={'eps':1e-09, 'disp': True, 'maxiter':200}, method='SLSQP',bounds=bnds)
        return result
    bounds = ((-0.9999999, 0.9999999),(2.1,50))
    result=Optimizer(mReturns, mUt, vTheta, LL_Copula_Student, bounds)
    return result.x, -result.fun, result.success

dLogLikelihood = gammaln((dNu+2)/2) + gammaln(dNu/2) - iDimension*gammaln((dNu+1)/2) - (dNu+iDimension)/2 * \
                 np.log(1+(vX.T@inv(mR)@vX)/dNu) + (dNu+1)/2*np.log(1+vX[0]**2/dNu) + \
                 (dNu+1)/2*np.log(1+vX[1]**2/dNu) 
dSum+=np.asscalar(dLogLikelihood)

In [10]:
(vTheta,LL_Copula_St,success_Copula_St)=Parameters_Copula_Student(np.array(mInSampleReturns).T,mUt)
print(vTheta)
print(LL_Copula_St)
print(success_Copula_St)

Optimization terminated successfully.    (Exit mode 0)
            Current function value: -1531.9034059941157
            Iterations: 33
            Function evaluations: 151
            Gradient evaluations: 30
[ 0.91677754  7.40781997]
1531.9034059941157
True


In [11]:
def AIC(vTheta1, vTheta2, vTheta3, dLL1, dLL2, dLL3):
    iK=len(vTheta1)+len(vTheta2)+len(vTheta3)
    dLL=dLL1+dLL2+dLL3
    dAIC= 2*iK-2*dLL
    print(dAIC)

In [12]:
AIC(parameter1_Stud,parameter2_Stud,vTheta, likelihood1_Stud, likelihood2_Stud , LL_Copula_St)

6376.609777268901


In [17]:
def LL_GARCH_Student(vTheta,vReturns):
    dOmega = vTheta[0]
    dAlpha = vTheta[1]
    dBeta  = vTheta[2]
    dNu = vTheta[3]
    iT=len(vReturns)
    vH=np.zeros(iT)

    for t in range(iT):
        if t == 0:
            vH[t] = np.var(vReturns) 
        else:
            vH[t] = dOmega + dAlpha*vReturns[t-1]**2 + dBeta * vH[t-1]    

    vLogPdf = gammaln( (dNu+1)/2 )-gammaln( dNu/2 ) - 0.5*np.log( np.pi*(dNu-2)*vH[1:] ) \
            - 0.5 * (dNu+1) * np.log( 1 + ( vReturns[1:]**2 / ( (dNu-2)*vH[1:] ) ) )
    return np.sum(vLogPdf)

In [18]:
    def LL_Copula_Student(vTheta,mReturns,mUt):
        dRho12 = vTheta[0]
        dNu = vTheta[1]
        iT = mReturns.shape[1]
        iDimension = len(mReturns)
        mQuantile= st.t.ppf(mUt, dNu, loc=np.zeros(mUt.shape), scale=np.ones(mUt.shape))
        mR = np.ones((iDimension,iDimension))
        mR[1,0] = dRho12
        mR[0,1] = dRho12
        
        dSum=0
        
        for t in range(1,iT):
            vX = mQuantile[:,t]
            vX = vX.reshape((iDimension,1))
            dLogf1=st.t.logpdf(vX[0],dNu)
            dLogf2=st.t.logpdf(vX[1],dNu)
#             dLogLikelihood = gammaln((dNu+2)/2) - gammaln(dNu/2) - 0.5*np.log((np.pi*dNu)**2) - 0.5 * np.log(det(mR)) \
#                             - (dNu+2)/2 * np.log(1+(vX.T @ inv(mR) @ vX)/dNu) - dLogf1-dLogf2
            dLogLikelihood = gammaln((dNu+2)/2) + gammaln(dNu/2) - iDimension * gammaln((dNu+1)/2) - 1/2 * np.log(det(mR)) \
                             -(dNu+iDimension)/2 *np.log(1+(vX.T@inv(mR)@vX)/dNu) + (dNu+1)/2 * np.log(1+vX[0]**2/dNu) + \
                             (dNu+1)/2* np.log(1+vX[1]**2/dNu)             
                
            dSum += np.asscalar(dLogLikelihood)
    
        return dSum

In [15]:
def LogarithmicScore(vTheta1,vTheta2,vTheta3,mOutSampleReturns):
    dLL1 = LL_GARCH_Student(vTheta1,np.array(mOutSampleReturns.iloc[:,0]))
    dLL2 = LL_GARCH_Student(vTheta2,np.array(mOutSampleReturns.iloc[:,1]))
    mReturns=np.array(mOutSampleReturns).T
    mVarianceOutSample = ComputeSigma2GARCH(vTheta1,vTheta2,mReturns)
    vUt1OutSample=st.t.cdf(mReturns[0,:], vTheta1[3], loc=np.zeros(mReturns.shape[1]), \
              scale=np.sqrt((vTheta1[3]-2)/vTheta1[3]*mVarianceOutSample[0,:]))
    vUt2OutSample=st.t.cdf(mReturns[1,:], vTheta2[3], loc=np.zeros(mReturns.shape[1]), \
              scale=np.sqrt((vTheta2[3]-2)/vTheta2[3]*mVarianceOutSample[1,:]))
    vUt1OutSample=vUt1OutSample.reshape((1,len(mOutSampleReturns)))
    vUt2OutSample=vUt2OutSample.reshape((1,len(mOutSampleReturns)))
    mUtOutSample=np.concatenate((vUt1OutSample, vUt2OutSample), axis=0)
    dLL3 = LL_Copula_Student(vTheta3,mReturns,mUtOutSample)
    dLogScore = dLL1+dLL2+dLL3
    
    return dLogScore

In [19]:
dLogScore = LogarithmicScore(parameter1_Stud,parameter2_Stud,vTheta,mOutSampleReturns)

In [20]:
print(dLogScore)

-1554.67180199
